### Import Libraries

In [2]:
import os

os.chdir('../scripts') # cd into scripts dir

from scan import *
from prep import *
import pandas as pd, numpy as np
import matplotlib.pyplot as plt

os.chdir('../') # cd back to parent dir
home_dir = "compas/" # set home dir for reading and storing data

if not os.path.exists("plots"):
    os.mkdir("plots")

Set flags to decide whether to generate plots for each experiment.

In [1]:
sim_1 = True
sim_2 = True
mit_1 = True
mit_2 = True

In [ ]:
vals = [0, 1, 3, 10]
compas_sim_1_df = pd.read_csv(home_dir + "datasets/compas_prep.csv")
compas_sim_1_df = compas_sim_1_df.loc[compas_sim_df['k'].isin(vals)]

group_df = compas_sim_df.groupby(['k', 'lambda']).mean().reset_index()
iou_df = group_df[['k', 'lambda', 'iou']]
score_df = group_df[['k', 'lambda', 'score']]

In [ ]:
# Plot score curves for each k
score_fig = plt.figure(figsize=(7.5, 6))
k_vals = list(set(list(group_df['k'])))
k_vals.sort()
for k in k_vals:
    score_k_df = score_df.loc[score_df['k'] == k]
    plt.plot(score_k_df['lambda'], score_k_df['score'], label='k={}'.format(k))

# plt.title("Score as a Function of Lambda", fontsize=24)
plt.xlabel("lambda", fontsize=20)
plt.ylabel("score", fontsize=20)
plt.legend(loc=1, prop={'size': 16})
score_fig.savefig(score_bs, format="png")
# plt.show()

In [ ]:
# Generate Theoretical Lambda Cutoffs
def gen_lambda_cutoff(k):
    if k == 0:
        return 50
    else:
        return (75/k)*(4999-k**2)/(7497-k**2)

In [ ]:
iou_df

In [ ]:
theo_0_df = pd.DataFrame(k_vals, columns=['k'])
theo_0_df['lambda'] = theo_0_df['k'].apply(lambda k : gen_lambda_cutoff(k))
theo_1_df = theo_0_df.copy(deep=True)
theo_0_df['iou'] = 0
theo_1_df['iou'] = 1
theo_df = theo_0_df.append(theo_1_df)
theo_df

In [ ]:
def get_color():
    return (np.random.rand(), np.random.rand(), np.random.rand())

In [ ]:
np.random.seed(1)
iou_fig = plt.figure(figsize=(10, 8))
k_vals = list(set(list(group_df['k'])))
k_vals.sort()
for k in k_vals:
    iou_k_df = iou_df.loc[iou_df['k'] == k]
    theo_k_df = theo_df.loc[theo_df['k'] == k]
    color = get_color()
    plt.plot(iou_k_df['lambda'], iou_k_df['iou'], label='Actual k={}'.format(k), color=color, marker='o')
    plt.plot(theo_k_df['lambda'], theo_k_df['iou'], label='Theo k={}'.format(k), color=color, linestyle='dashed')

# plt.title("Detection: Actual vs. Theoretical Lambda Cutoffs", fontsize=24)
plt.xlabel("lambda", fontsize=20)
plt.ylabel("iou", fontsize=20)
plt.legend(loc=1, prop={'size': 16})
io_bs = io.BytesIO()
iou_fig.savefig(io_bs, format="png")
folder.upload_stream("iou_" + sim + ".png", io_bs.getvalue())
# plt.show()

In [ ]:
# Recipe outputs
ja4hiuwi = dataiku.Folder("ja4hiuWI")
ja4hiuwi_info = ja4hiuwi.get_info()